In [5]:
%load_ext autoreload
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize client
client = genai.Client(api_key=os.getenv("GOOGLE_GENERATIVE_AI_API_KEY"))

import os
import json

video_json_path = "myfile.json"

# If the file exists, load it; otherwise, upload and save
if os.path.exists(video_json_path):
    with open(video_json_path, "r") as f:
        myfile_dict = json.load(f)
    # Reconstruct a File object if needed, or just use the dict
    myfile = types.File(**myfile_dict)
    print(f"Video loaded from cache: {myfile.name}")
else:
    # myfile = client.files.upload(file="videos/output_long_again_2.mp4")
    myfile = client.files.upload(file="videos/output_long_again_2_timestamped.mp4")
    print(f"Video uploaded: {myfile.name}")
    # Save the model dump to JSON
    with open(video_json_path, "w") as f:
        f.write(myfile.model_dump_json())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Video loaded from cache: files/p3d949zrnvip


In [9]:
# STEP 1: PROCEDURE EXTRACTION (runs first)
from video_understanding.multi_prompts import create_procedure_extraction_prompt
from video_understanding.simple_models import ProcedureExtraction

procedure_json_path = "procedure_result.json"

print("=" * 50)
print("STEP 1: PROCEDURE EXTRACTION")
print("=" * 50)

system_prompt, user_prompt = create_procedure_extraction_prompt()

contents = [myfile, user_prompt]
print(f"Token count: {client.models.count_tokens(model='models/gemini-2.5-pro', contents=contents)}")

procedure_response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=contents,
    config={
        "response_mime_type": "application/json",
        "response_schema": ProcedureExtraction,
        "system_instruction": system_prompt
    },
)


procedure_result = procedure_response.parsed
with open(procedure_json_path, "w") as f:
    f.write(procedure_result.model_dump_json())
print("PROCEDURE EXTRACTION COMPLETE")
print(f"Goal wells: {len(procedure_result.goal_wells)}")
print(f"Reagent sources: {procedure_result.reagent_sources}")
print(f"Timestamp range: {procedure_result.timestamp_range}")

STEP 1: PROCEDURE EXTRACTION
Token count: total_tokens=19668 cached_content_token_count=None
PROCEDURE EXTRACTION COMPLETE
Goal wells: 3
Reagent sources: ['A', 'B', 'C']
Timestamp range: 0:00 - 1:12


In [ ]:
import os
import json
from video_understanding.simple_models import ProcedureExtraction

procedure_json_path = "procedure_result.json"

# Try to load it back in if it exists
if os.path.exists(procedure_json_path):
    with open(procedure_json_path, "r") as f:
        loaded_json = f.read()
    procedure_result = ProcedureExtraction.model_validate_json(loaded_json)
    print("Loaded ProcedureExtraction from file:")
    print(f"Goal wells: {len(procedure_result.goal_wells)}")
    print(f"Reagent sources: {procedure_result.reagent_sources}")
    print(f"Timestamp range: {procedure_result.timestamp_range}")
else:
    print("ProcedureExtraction file not found.")

In [ ]:
# STEP 2: OBJECTIVE EVENTS EXTRACTION
from video_understanding.multi_prompts import create_objective_events_prompt
from video_understanding.simple_models import ObjectiveEventsList
from pydantic import BaseModel, Field
from typing import List, Union


objective_events_json_path = "objective_events_result.json"

print("=" * 50)
print("STEP 2: OBJECTIVE EVENTS EXTRACTION")
print("=" * 50)

system_prompt, user_prompt = create_objective_events_prompt(procedure_result)

contents = [myfile, user_prompt]
print(f"Token count: {client.models.count_tokens(model='models/gemini-2.5-pro', contents=contents)}")



objective_response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=contents,
    config={
        "response_mime_type": "application/json",
        "response_schema": ObjectiveEventsList,
        "system_instruction": system_prompt
    },
)

objective_events_result = objective_response.parsed
objective_events = objective_events_result.events

with open(objective_events_json_path, "w") as f:
    f.write(objective_events_result.model_dump_json())

print(f"OBJECTIVE EVENTS EXTRACTION COMPLETE")
print(f"Total events found: {len(objective_events)}")

# Count by type
event_counts = {}
for event in objective_events:
    event_type = type(event).__name__
    event_counts[event_type] = event_counts.get(event_type, 0) + 1

for event_type, count in event_counts.items():
    print(f"- {event_type}: {count}")

In [ ]:
import os
import json
from video_understanding.simple_models import ObjectiveEventsList

objective_events_json_path = "objective_events_result.json"

# Try to load it back in if it exists
if os.path.exists(objective_events_json_path):
    with open(objective_events_json_path, "r") as f:
        loaded_json = f.read()
    objective_events_result = ObjectiveEventsList.model_validate_json(loaded_json)
    print("Loaded ObjectiveEventsList from file:")
    print(f"Total events found: {len(objective_events_result.events)}")
    event_counts = {}
    for event in objective_events_result.events:
        event_type = type(event).__name__
        event_counts[event_type] = event_counts.get(event_type, 0) + 1
    for event_type, count in event_counts.items():
        print(f"- {event_type}: {count}")
else:
    print("ObjectiveEventsList file not found.")

In [ ]:
# STEP 3: ANALYSIS EVENTS EXTRACTION
from video_understanding.multi_prompts import create_analysis_events_prompt
from video_understanding.simple_models import AnalysisEventsResult


print("=" * 50)
print("STEP 3: ANALYSIS EVENTS EXTRACTION")
print("=" * 50)

system_prompt, user_prompt = create_analysis_events_prompt(procedure_result, objective_events_result)

contents = [myfile, user_prompt]
print(f"Token count: {client.models.count_tokens(model='models/gemini-2.5-pro', contents=contents)}")


analysis_response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=contents,
    config={
        "response_mime_type": "application/json",
        "response_schema": AnalysisEventsResult,
        "system_instruction": system_prompt
    },
)

analysis_events = analysis_response.parsed
print("ANALYSIS EVENTS EXTRACTION COMPLETE")

In [7]:
with open("analysis_events_result.json", "w") as f:
    f.write(analysis_response.parsed.model_dump_json())